In [55]:
import spacy
from spacy.tokens import Doc
import re

In [56]:
coref_model = spacy.load("en_coreference_web_trf")
doc = coref_model("At Bosch, we do things differently. The company intends to increase workforce diversity by 40%") 

def resolve_references(doc):
    token_mention_mapper = {}
    output_string = ""
    clusters = [val for key, val in doc.spans.items() if key.startswith("coref_cluster")]

    for cluster in clusters:
        first_mention = cluster[0]
        for mention_span in list(cluster)[1:]:
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
            
            for token in mention_span[1:]:
                token_mention_mapper[token.idx] = ""
    for token in doc:
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        else:
            output_string += token.text + token.whitespace_

    return output_string

resolved_string = resolve_references(doc)
print(resolved_string)

At Bosch, Bosch do things differently. Bosch intends to increase workforce diversity by 40%


In [57]:
# Perform standard imports
import spacy
nlp = spacy.load('en_core_web_trf')
ner_op_string = resolved_string

# Write a function to display basic entity info:
def show_ents(doc):
    global ner_op_string
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_)))
            if ent.label_ == 'ORG':
                word_length = ent.end_char - ent.start_char
                ner_op_string = ner_op_string[:ent.start_char] + 'O'*(word_length) + ner_op_string[ent.end_char:]
    else:
        print('No named entities found.')
    return ner_op_string
    
doc = nlp(resolved_string)
show_ents(doc)
ner_op_string = re.sub(r'O{2,}', 'ORG',ner_op_string)
ner_op_string

Bosch - 3 - 8 - ORG - Companies, agencies, institutions, etc.
Bosch - 10 - 15 - ORG - Companies, agencies, institutions, etc.
Bosch - 39 - 44 - ORG - Companies, agencies, institutions, etc.
40% - 88 - 91 - PERCENT - Percentage, including "%"


'At ORG, ORG do things differently. ORG intends to increase workforce diversity by 40%'